In [27]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import torch
from torch import nn, optim
from torch.utils import data
import torchvision
from torchvision import datasets, transforms

from livelossplot import PlotLosses

In [28]:
torch.manual_seed(42)

In [29]:
classes = ["apple", "spider", "octopus", "snowflake"]

In [30]:
limit=500

In [31]:
images = []

for label in classes:
    X = np.load('google_draw/{}.npy'.format(label))
    print("Loaded {} out of {} {}s".format(limit, X.shape[0], label))
    images.append(X[:limit])

X = np.concatenate(images)
Y = np.concatenate([limit * [i] for i in range(len(classes))])

Loaded 500 out of 144722 apples
Loaded 500 out of 209447 spiders
Loaded 500 out of 150152 octopuss
Loaded 500 out of 116685 snowflakes


In [32]:
print("X shape: {}, type:{}".format(X.shape,X.dtype))

X shape: (2000, 784), type:uint8


In [33]:
X = X.reshape(-1, 1, 28, 28)
X = X.astype('float32') / 255.
X = np.pad(X, [(0, 0), (0, 0), (2, 2), (2, 2)], mode='constant')

# (samples, channels, x, y)
X.shape

(2000, 1, 32, 32)

In [34]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [35]:
# define data loaders
dataloaders = {
    'train':
    data.DataLoader(
        data.TensorDataset(torch.from_numpy(X_train), torch.from_numpy(Y_train).long()),
        batch_size=64,
        shuffle=True, num_workers=4
    ),
    'validation': 
    data.DataLoader(
        data.TensorDataset(torch.from_numpy(X_test), torch.from_numpy(Y_test).long()),
        batch_size=64,
        shuffle=False, num_workers=4
    )
}

In [36]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [37]:
device

device(type='cuda', index=0)

In [38]:
def train_model(model, criterion, optimizer, num_epochs=10):
    liveloss = PlotLosses()
    model = model.to(device)
    
    for epoch in range(num_epochs):
        logs = {}
        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += (preds == labels.data).sum().item()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects / len(dataloaders[phase].dataset)
            
            prefix = ''
            if phase == 'validation':
                prefix = 'val_'

            logs[prefix + 'log loss'] = epoch_loss
            logs[prefix + 'accuracy'] = epoch_acc
        
        liveloss.update(logs)
        liveloss.draw()
    return model

In [39]:
# antipattern in PyTorch, don't do it!
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

In [90]:
class PrimeNet(nn.Module):
    def __init__(self, activation_fun, reduction_fun channels_out_num, conv_count, conv_block_count,linear_count):
        super().__init__()
        self.activation_fun=activation_fun
        self.conv=[]
        ch_in=1
        ch_out=channels_out_num/(2**(conv_count-1))
        for i in range(conv_count): 
            self.conv.append(self._conv_block(int(ch_in),int(ch_out)))
            ch_in=ch_out
            ch_out*=2
        
        self.fc = nn.Sequential(
            Flatten(),
            nn.Linear(int(ch_out/2 * ((32/(2**conv_count)) ** 2)), 128),
            activation_fun(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(128, len(classes))
        )
    
    def _conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            self.activation_fun(inplace=True),
            reduction_fun(2, 2)
        )
    
    def forward(self, x):
        for conv in self.conv:
            x=conv(x)
        x = self.fc(x)
        return x

In [91]:
model1=PrimeNet(nn.ReLU, 32,2)
criterion = nn.CrossEntropyLoss()
optimizer1 = optim.Adam(model1.parameters(), lr=1e-4)
epoch_num=20

In [92]:
model1_trained = train_model(model1, criterion, optimizer1, num_epochs=epoch_num)

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same